In [1]:
import pandas as pd

In [2]:
report_content_filepath = '/home/faith/datasets/mimic_512/training.csv'
reports_csv = pd.read_csv(report_content_filepath)

In [3]:
dir = reports_csv['image_path'].apply(lambda row: row.split('/'))
dir.head()

0    [files, p10, p10000032, s50414267, 02aa804e-bd...
1    [files, p10, p10000032, s50414267, 174413ec-4e...
2    [files, p10, p10000032, s53189527, 2a2277a9-b0...
3    [files, p10, p10000032, s53189527, e084de3b-be...
4    [files, p10, p10000032, s53911762, 68b5c4b1-22...
Name: image_path, dtype: object

In [4]:
reports_csv['pg'] = dir.apply(lambda row: row[1])
reports_csv['pName'] = dir.apply(lambda row: row[2])
reports_csv['sName'] = dir.apply(lambda row: row[3])

In [5]:
reports_csv.head()

,Unnamed: 0,image_path,report_content,pg,pName,sName
0,0,files/p10/p10000032/s50414267/02aa804e-bde0afd...,FINAL REPORT EXAMINATION : CHEST ( PA AND LAT ...,p10,p10000032,s50414267
1,1,files/p10/p10000032/s50414267/174413ec-4ec4c1f...,FINAL REPORT EXAMINATION : CHEST ( PA AND LAT ...,p10,p10000032,s50414267
2,2,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,FINAL REPORT EXAMINATION : CHEST ( PA AND LAT ...,p10,p10000032,s53189527
3,3,files/p10/p10000032/s53189527/e084de3b-be89b11...,FINAL REPORT EXAMINATION : CHEST ( PA AND LAT ...,p10,p10000032,s53189527
4,4,files/p10/p10000032/s53911762/68b5c4b1-227d048...,FINAL REPORT EXAMINATION : CHEST ( PORTABLE AP...,p10,p10000032,s53911762


In [6]:
def foo(row):
    print(row)
    return row

In [7]:
test = reports_csv['image_path'].head().apply(lambda row: foo(row))

files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg
files/p10/p10000032/s50414267/174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962.jpg
files/p10/p10000032/s53189527/2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab.jpg
files/p10/p10000032/s53189527/e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c.jpg
files/p10/p10000032/s53911762/68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714.jpg


In [8]:
test_dict = { ('hello', 'hi'): 1 }

In [9]:
reports_csv['key'] = reports_csv.apply(lambda row: (row[4], row[5]), axis=1)

In [10]:
reports_csv['key'].dtype

dtype('O')

In [11]:
reports_dict = dict(zip(reports_csv['key'], reports_csv['report_content']))
reports_dict

{('p10000032',
  's50414267'): 'FINAL REPORT EXAMINATION : CHEST ( PA AND LAT ) INDICATION : ___F with new onset ascites / / eval for infection TECHNIQUE : Chest PA and lateral COMPARISON : None . FINDINGS : There is no focal consolidation , pleural effusion or pneumothorax . Bilateral nodular opacities that most likely represent nipple shadows . The cardiomediastinal silhouette is normal . Clips project over the left lung , potentially within the breast . The imaged upper abdomen is unremarkable . Chronic deformity of the posterior left sixth and seventh ribs are noted . IMPRESSION : No acute cardiopulmonary process .',
 ('p10000032',
  's53189527'): 'FINAL REPORT EXAMINATION : CHEST ( PA AND LAT ) INDICATION : History : ___F with shortness of breath TECHNIQUE : Chest PA and lateral COMPARISON : ___ FINDINGS : The cardiac , mediastinal and hilar contours are normal . Pulmonary vasculature is normal . Lungs are clear . No pleural effusion or pneumothorax is present . Multiple clips are

In [13]:
example_report = reports_dict[('p10000032','s50414267')]
print(example_report)

FINAL REPORT EXAMINATION : CHEST ( PA AND LAT ) INDICATION : ___F with new onset ascites / / eval for infection TECHNIQUE : Chest PA and lateral COMPARISON : None . FINDINGS : There is no focal consolidation , pleural effusion or pneumothorax . Bilateral nodular opacities that most likely represent nipple shadows . The cardiomediastinal silhouette is normal . Clips project over the left lung , potentially within the breast . The imaged upper abdomen is unremarkable . Chronic deformity of the posterior left sixth and seventh ribs are noted . IMPRESSION : No acute cardiopulmonary process .


In [16]:
import re

In [21]:
def parse_report(report):

    #helper regex functions
    cached = {}

    def get(pattern, flags=0):
        key = hash((pattern, flags))
        if key not in cached:
            cached[key] = re.compile(pattern, flags=flags)

        return cached[key]

    def sub(pattern, repl, string, flags=0):
        return get(pattern, flags=flags).sub(repl, string)

    def rm(pattern, string, flags=0):
        return sub(pattern, '', string)

    def get_id(tag, flags=0):
        return get(r'\[\*\*.*{:s}.*?\*\*\]'.format(tag), flags=flags)

    def sub_id(tag, repl, string, flags=0):
        return get_id(tag).sub(repl, string)

    #report parsing logic
    report = report.lower()
    report = sub_id(r'(?:location|address|university|country|state|unit number)', 'LOC', report)
    report = sub_id(r'(?:year|month|day|date)', 'DATE', report)
    report = sub_id(r'(?:hospital)', 'HOSPITAL', report)
    report = sub_id(r'(?:identifier|serial number|medical record number|social security number|md number)',
                             'ID', report)
    report = sub_id(r'(?:age)', 'AGE', report)
    report = sub_id(r'(?:phone|pager number|contact info|provider number)', 'PHONE', report)
    report = sub_id(r'(?:name|initial|dictator|attending)', 'NAME', report)
    report = sub_id(r'(?:company)', 'COMPANY', report)
    report = sub_id(r'(?:clip number)', 'CLIP_NUM', report)

    report = sub((
        r'\[\*\*(?:'
        r'\d{4}'  # 1970
        r'|\d{0,2}[/-]\d{0,2}'  # 01-01
        r'|\d{0,2}[/-]\d{4}'  # 01-1970
        r'|\d{0,2}[/-]\d{0,2}[/-]\d{4}'  # 01-01-1970
        r'|\d{4}[/-]\d{0,2}[/-]\d{0,2}'  # 1970-01-01
        r')\*\*\]'
    ), 'DATE', report)
    report = sub(r'\[\*\*.*\*\*\]', 'OTHER', report)
    report = sub(r'(?:\d{1,2}:\d{2})', 'TIME', report)

    report = rm(r'_{2,}', report, flags=re.MULTILINE)
    report = rm(r'the study and the report were reviewed by the staff radiologist.', report)

    matches = list(get(r'^(?P<title>[ \w()]+):', flags=re.MULTILINE).finditer(report)) #Remove carat and newline flag
    parsed_report = {}

    for (match, next_match) in zip(matches, matches[1:] + [None]):
        start = match.end()
        end = next_match and next_match.start()

        title = match.group('title')
        title = title.strip()

        paragraph = report[start:end]
        paragraph = sub(r'\s{2,}', ' ', paragraph)
        paragraph = paragraph.strip()

        parsed_report[title] = paragraph.replace('\n', '\\n')

    print(parsed_report)

In [22]:
parse_report(example_report)

{'final report examination': 'chest ( pa and lat ) indication : f with new onset ascites / / eval for infection technique : chest pa and lateral comparison : none . findings : there is no focal consolidation , pleural effusion or pneumothorax . bilateral nodular opacities that most likely represent nipple shadows . the cardiomediastinal silhouette is normal . clips project over the left lung , potentially within the breast . the imaged upper abdomen is unremarkable . chronic deformity of the posterior left sixth and seventh ribs are noted . impression : no acute cardiopulmonary process .'}
